In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from streamtasks.comm.types import *
import fastavro

PricedIdSchema = fastavro.parse_schema({
    "type": "record",
    "name": "PricedId",
    "fields": [
        {"name": "id", "type": "int"},
        {"name": "cost", "type": "int"}
    ]
})

SCHEMA_ID_MAP = {
    TopicDataMessage: 0,
    TopicControlMessage: 1,
    AddressedMessage: 2,
    AddressesChangedMessage: 3,
    InTopicsChangedMessage: 4,
    OutTopicsChangedMessage: 5,
}

SCHEMA_MAP = {
    0: fastavro.parse_schema({
        "type": "record",
        "name": "TopicDataMessage",
        "fields": [
            {"name": "topic", "type": "int"},
            {"name": "data", "type": "bytes"}
        ]
    }),
    1: fastavro.parse_schema({
        "type": "record",
        "name": "TopicControlMessage",
        "fields": [
            {"name": "topic", "type": "int"},
            {"name": "paused", "type": "boolean"}
        ]
    }),
    2: fastavro.parse_schema({
        "type": "record",
        "name": "AddressedMessage",
        "fields": [
            {"name": "address", "type": "int"},
            {"name": "data", "type": "bytes"}
        ]
    }),
    3: fastavro.parse_schema({
        "type": "record",
        "name": "AddressesChangedMessage",
        "fields": [
            {"name": "add", "type": {"type": "array", "items": PricedIdSchema}},
            {"name": "remove", "type": {"type": "array", "items": "int"}}
        ]
    }),
    4: fastavro.parse_schema({
        "type": "record",
        "name": "InTopicsChangedMessage",
        "fields": [
            {"name": "add", "type": {"type": "array", "items": "int"}},
            {"name": "remove", "type": {"type": "array", "items": "int"}}
        ]
    }),
    5: fastavro.parse_schema({
        "type": "record",
        "name": "OutTopicsChangedMessage",
        "fields": [
            {"name": "add", "type": {"type": "array", "items": PricedIdSchema}},
            {"name": "remove", "type": {"type": "array", "items": "int"}}
        ]
    }),
}

In [18]:
import struct

struct.pack('<B', 0)

b'\x00'

In [33]:
from io import BytesIO

def serialize_message(message: Message) -> bytes:
    stream = BytesIO()
    id = SCHEMA_ID_MAP[type(message)]
    stream.write(struct.pack("<B", id))
    schema = SCHEMA_MAP[id]
    fastavro.schemaless_writer(message, schema, stream)
    return stream.getvalue()

def deserialize_message(data: bytes) -> Message:
    stream = BytesIO(data)
    id = struct.unpack("<B", stream.read(1))[0]
    schema = SCHEMA_MAP[id]
    return fastavro.schemaless_reader(stream, schema)

In [34]:
deserialize_message(serialize_message(TopicDataMessage(1, b'hello')))

ValueError: no value and no default for topic